# Swiss rail network map

Visualize the full Swiss rail network graph on top of an interactive basemap.

In [1]:
from pathlib import Path
import pickle

import networkx as nx
import pandas as pd

GRAPH_PATH = Path('datasets/switzerland/swiss_rail_network.gpickle')
STATION_METADATA_PATH = Path('datasets/switzerland/sbb-dienststellen-gemass-opentransportdataswiss.csv')

with GRAPH_PATH.open('rb') as f:
    G = pickle.load(f)

station_metadata = pd.read_csv(STATION_METADATA_PATH, sep=';')
stop_point_mask = station_metadata['stopPoint'].astype(str).str.lower() == 'true'
station_metadata['abbreviation_clean'] = (
    station_metadata['abbreviation']
    .astype(str)
    .str.strip()
    .str.upper()
)
station_abbreviation_set = set(
    station_metadata.loc[stop_point_mask, 'abbreviation_clean'].dropna().tolist()
)
station_abbreviation_set.discard('NAN')

print(f"Loaded graph with {G.number_of_nodes():,} nodes and {G.number_of_edges():,} edges")
print(f"Passenger stop abbreviations with timetable data: {len(station_abbreviation_set):,}")

Loaded graph with 1,355 nodes and 1,512 edges
Passenger stop abbreviations with timetable data: 3,211


In [2]:
def parse_geopos(value):
    if isinstance(value, str) and ',' in value:
        lat_str, lon_str = value.split(',', 1)
        try:
            lat = float(lat_str.strip())
            lon = float(lon_str.strip())
        except ValueError:
            return None
        if -90 <= lat <= 90 and -180 <= lon <= 180:
            return lat, lon
    return None

def node_is_station(node_id: str) -> bool:
    return node_id.strip().upper() in station_abbreviation_set

node_records = []
missing_coords = []
station_nodes = 0
infra_nodes = 0
for node, data in G.nodes(data=True):
    coords = None
    for row in data.get('rows', []):
        coords = parse_geopos(row.get('Geopos')) or parse_geopos(row.get('Geopos_didok'))
        if coords:
            break
    if coords is None:
        missing_coords.append(node)
        continue
    stop_names = [name for name in data.get('stop_names', []) if isinstance(name, str)]
    lines = sorted(set(data.get('lines', [])))
    is_station_node = node_is_station(node)
    station_nodes += int(is_station_node)
    infra_nodes += int(not is_station_node)
    node_records.append({
        'node_id': node,
        'label': stop_names[0] if stop_names else node,
        'lat': coords[0],
        'lon': coords[1],
        'is_station': is_station_node,
        'stop_names': stop_names,
        'lines': lines,
    })

node_positions_df = pd.DataFrame(node_records)
positions = {rec['node_id']: (rec['lat'], rec['lon']) for rec in node_records}
print(f"Nodes with coordinates: {len(node_positions_df)} / {G.number_of_nodes()}")
print(f"Missing coordinates for {len(missing_coords)} nodes")
print(f"Station nodes plotted: {station_nodes:,} | Infrastructure nodes plotted: {infra_nodes:,}")

Nodes with coordinates: 1355 / 1355
Missing coordinates for 0 nodes
Station nodes plotted: 765 | Infrastructure nodes plotted: 590


In [3]:
import folium

if node_positions_df.empty:
    raise ValueError('No node positions available for mapping.')

center_lat = node_positions_df['lat'].mean()
center_lon = node_positions_df['lon'].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=8, tiles='CartoDB Positron')

edges_fg = folium.FeatureGroup(name='Rail segments', show=False)
for u, v in G.edges():
    if u not in positions or v not in positions:
        continue
    coords = [positions[u], positions[v]]
    folium.PolyLine(coords, color='#6c757d', weight=1, opacity=0.6).add_to(edges_fg)
edges_fg.add_to(m)

stations_fg = folium.FeatureGroup(name='Stations')
infra_fg = folium.FeatureGroup(name='Infrastructure nodes')
for rec in node_records:
    layer = stations_fg if rec['is_station'] else infra_fg
    color = '#1f77b4' if rec['is_station'] else '#ff7f0e'
    popup_lines = [
        f"<b>{rec['label']}</b> ({rec['node_id']})",
        f"Type: {'Station' if rec['is_station'] else 'Infrastructure node'}",
    ]
    if rec['stop_names']:
        popup_lines.append('Stops: ' + ', '.join(rec['stop_names']))
    if rec['lines']:
        popup_lines.append('Lines: ' + ', '.join(map(str, rec['lines'])))
    popup_html = '<br>'.join(popup_lines)
    folium.CircleMarker(
        location=[rec['lat'], rec['lon']],
        radius=4 if rec['is_station'] else 3,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.9,
        weight=1,
        tooltip=rec['label'],
        popup=folium.Popup(popup_html, max_width=260),
    ).add_to(layer)

stations_fg.add_to(m)
infra_fg.add_to(m)
folium.LayerControl(collapsed=False).add_to(m)
m